In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context


## IBM Data Science Cap Stone Project Final Report

In this notebook, I will be creating clusters to find the most suitable location live for people who don't have cars in metro vancouer, canada.

In [2]:
#importing necessary libraries
import requests
import lxml.html as lh
import pandas as pd
import re

In [3]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [4]:
#https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_row = doc.xpath('//tr//td')

In [5]:
#Check the length of the first 12 rows
[len(T) for T in tr_row[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [6]:
#Create empty list
col=[]
col.append(("postalCode",[]))
col.append(("neighbourhood",[]))
col.append(("community",[]))


print(col)
print(col[0])
print(col[0][1])

[('postalCode', []), ('neighbourhood', []), ('community', [])]
('postalCode', [])
[]


In [7]:
#Since out first row is the header, data is stored on the second row onwards
print(len(tr_row))
for j in range(1,len(tr_row)):
    #T is our j'th row
    T=tr_row[j]
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    data = T[0].text_content()
    col[0][1].append(data)
    data = T[2].text_content()
    split=re.split(r'\((.*)\)',data)
    col[1][1].append(split[0])
    if len(split)>1:
        col[2][1].append(split[1])
    else:
        col[2][1].append("")

235


In [8]:
[len(C) for (title,C) in col]

[179, 179, 179]

In [9]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
print(df)

    postalCode                  neighbourhood  \
0          V2A                      Penticton   
1          V3A               Langley Township   
2          V4A                SurreySouthwest   
3          V5A                        Burnaby   
4          V6A                      Vancouver   
5          V7A                  RichmondSouth   
6          V8A                   Powell River   
7          V9A                       Victoria   
8          V1B                     VernonEast   
9          V2B              KamloopsNorthwest   
10         V3B          Port CoquitlamCentral   
11         V4B                     White Rock   
12         V5B                        Burnaby   
13         V6B                      Vancouver   
14         V7B                       Richmond   
15         V8B                       Squamish   
16         V9B                       Victoria   
17         V1C                      Cranbrook   
18         V2C  KamloopsCentral and Southeast   
19         V3C      

In [10]:
#to make it easier, we will store this in csv format.
#Export to .CSV
#df.to_csv('Vancouver_Postcodes.csv')
project.save_data(data=df.to_csv(index=False),file_name="Vancouver_Postcodes.csv",overwrite=True)


{'file_name': 'Vancouver_Postcodes.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstone-donotdelete-pr-gyrtxtz7k1quxs',
 'asset_id': '7b281d0f-e167-4a73-bb8d-b26010ef0264'}

In [11]:
!pip install geocoder
import numpy as np
import geocoder

     |████████████████████████████████| 102kB 11.8MB/s ta 0:00:01


In [12]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Vancouver, BC'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('V5R')

[49.23933500000004, -123.04110537999998]

In [13]:
postal_codes = df['postalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [14]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [15]:
# Check coordinates for a couple of neighborhoods
df[(df['postalCode']=='V5R') |(df['postalCode']=='V5E') ]

,postalCode,neighbourhood,community,Latitude,Longitude
30,V5E,Burnaby,Lakeview-Mayfield / Richmond Park / Kingsway-B...,49.226625,-122.954117
111,V5R,Vancouver,South Renfrew-Collingwood,49.239335,-123.041105


In [16]:
#Export to .CSV
#df.to_csv('Vancouver_Postcodes_LatLong.csv')
# Save dataframe as csv file to storage
project.save_data(data=df.to_csv(index=False),file_name='Vancouver_Postcodes_LatLong.csv',overwrite=True)

{'file_name': 'Vancouver_Postcodes_LatLong.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstone-donotdelete-pr-gyrtxtz7k1quxs',
 'asset_id': '623d0a56-e520-41a3-9c55-735f09ca464a'}

In [17]:
!pip install folium
import folium
from sklearn.cluster import KMeans

     |████████████████████████████████| 92kB 15.7MB/s eta 0:00:01


In [18]:
# Read .csv file from above
#df = pd.read_csv('Vancouver_Postcodes_LatLong.csv', index_col=0)
my_file = project.get_file('Vancouver_Postcodes_LatLong.csv')
pd.read_csv(my_file)
df.head()

,postalCode,neighbourhood,community,Latitude,Longitude
0,V2A,Penticton,,49.260380,-123.113360
1,V3A,Langley Township,Langley City,49.079040,-122.641921
2,V4A,SurreySouthwest,,49.045900,-122.954006
3,V5A,Burnaby,Government Road / Lake City / SFU / Burnaby Mo...,49.271170,-122.926292
4,V6A,Vancouver,Strathcona / Chinatown / Downtown Eastside,49.278215,-123.089651


In [19]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_vancouver = df['Latitude'].mean()
lon_vancouver = df['Longitude'].mean()
print('The geographical coordinates of Vancouver are {}, {}'.format(lat_vancouver, lon_vancouver))

The geographical coordinates of Vancouver are 49.241581213877076, -123.04638222750818


In [20]:
neighbourhoods = df['neighbourhood']
neighbourhood_color = {}
for i in neighbourhoods:
    neighbourhood_color[i]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [21]:
map_vancouver = folium.Map(location=[lat_vancouver, lon_vancouver], zoom_start=12)

# add markers to map
for lat, lng, neighbourhood, community in zip(df['Latitude'], 
                                           df['Longitude'],
                                           df['neighbourhood'], df['community']):
    label_text = label = '{}, {}'.format(neighbourhood, community)
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=neighbourhood_color[neighbourhood],
        fill_color=neighbourhood_color[neighbourhood],
        fill_opacity=0.7).add_to(map_vancouver)  
    
map_vancouver

## Getting Venues Data using Foursquare

In [22]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [23]:
def getNearbyVenues(postcodes, names, communities, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for postcode, name, community, lat, lng in zip(postcodes, names, communities, latitudes, longitudes):
        print(community)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(postcode,
            name, 
            community,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode','Neighborhood', 
                             'Community',
                              'Neighborhood Latitude', 
                              'Neighborhood Longitude', 
                              'Venue', 
                              'Venue Latitude', 
                              'Venue Longitude', 
                              'Venue Category']
    
    return(nearby_venues)

In [24]:
#Get venues for all neighborhoods in our dataset
vancouver_venues = getNearbyVenues(postcodes = df['postalCode'],names=df['neighbourhood'], communities=df['community'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude'])


[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4f77e130e4b0b009eebc0f50', 'name': "Yuk Yuk's Comedy Club", 'location': {'address': '2837 Cambie St.', 'crossStreet': '12 Ave.', 'lat': 49.26004488361375, 'lng': -123.11499826753109, 'labeledLatLngs': [{'label': 'display', 'lat': 49.26004488361375, 'lng': -123.11499826753109}], 'distance': 124, 'postalCode': 'V5Z 3X7', 'cc': 'CA', 'city': 'Vancouver', 'state': 'BC', 'country': 'Canada', 'formattedAddress': ['2837 Cambie St. (12 Ave.)', 'Vancouver BC V5Z 3X7', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d18e941735', 'name': 'Comedy Club', 'pluralName': 'Comedy Clubs', 'shortName': 'Comedy Club', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/comedyclub_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4f77e130e4b0b009eebc0f50-0'}, {'reasons': {'count': 

In [25]:
#Check size of resulting dataframe
vancouver_venues.shape

(594, 9)

In [26]:
vancouver_venues.head()

,Postcode,Neighborhood,Community,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,V2A,Penticton,,49.260380,-123.113360,Yuk Yuk's Comedy Club,49.260045,-123.114998,Comedy Club
1,V2A,Penticton,,49.260380,-123.113360,Kirin Seafood Restaurant 玉麒麟海鮮酒家 (Kirin Seafoo...,49.260866,-123.115421,Chinese Restaurant
2,V2A,Penticton,,49.260380,-123.113360,Firefly Fine Wines and Ales,49.259820,-123.114989,Liquor Store
3,V2A,Penticton,,49.260380,-123.113360,Share Tea,49.260030,-123.114870,Bubble Tea Shop
4,V6A,Vancouver,Strathcona / Chinatown / Downtown Eastside,49.278215,-123.089651,MacLean Park,49.278809,-123.088546,Park


In [27]:
project.save_data(data=vancouver_venues.to_csv(index=False),file_name='Vancouver_Venues.csv',overwrite=True)

{'file_name': 'Vancouver_Venues.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstone-donotdelete-pr-gyrtxtz7k1quxs',
 'asset_id': 'f963cf38-ae90-426f-8d85-ae493a64e61d'}

In [28]:
#Number of venues per neighborhood
vancouver_venues.groupby('Neighborhood').count()

,Postcode,Community,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,,
AbbotsfordEast,4,4,4,4,4,4,4,4
AbbotsfordSoutheast,4,4,4,4,4,4,4,4
AbbotsfordSouthwest,4,4,4,4,4,4,4,4
AbbotsfordWest,4,4,4,4,4,4,4,4
Burnaby,16,16,16,16,16,16,16,16
Campbell RiverCentral,4,4,4,4,4,4,4,4
Campbell RiverOutskirts,4,4,4,4,4,4,4,4
Castlegar,4,4,4,4,4,4,4,4
Central Saanich,4,4,4,4,4,4,4,4


In [29]:
#Number of venues per Venue Category
vancouver_venues.groupby('Venue Category').count().sort_values (by = 'Postcode')

,Postcode,Neighborhood,Community,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,,,
Filipino Restaurant,1,1,1,1,1,1,1,1
Cosmetics Shop,1,1,1,1,1,1,1,1
Steakhouse,1,1,1,1,1,1,1,1
Stadium,1,1,1,1,1,1,1,1
Dim Sum Restaurant,1,1,1,1,1,1,1,1
Inn,1,1,1,1,1,1,1,1
Electronics Store,1,1,1,1,1,1,1,1
Farm,1,1,1,1,1,1,1,1
Speakeasy,1,1,1,1,1,1,1,1


In [30]:
#Number of venues per community
vancouver_venues.groupby('Community').count()

,Postcode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Community,,,,,,,,
,317,317,317,317,317,317,317,317
Bentall Centre,17,17,17,17,17,17,17,17
Cascade-Schou / Douglas-Gilpin,3,3,3,3,3,3,3,3
Cedar,4,4,4,4,4,4,4,4
Central Kitsilano / Greektown,6,6,6,6,6,6,6,6
Colwood / South Langford / Metchosin,4,4,4,4,4,4,4,4
Cook Street Village / West Fairfield / Harris Green / Humboldt Valley / James Bay,4,4,4,4,4,4,4,4
Downtown / Chinatown,4,4,4,4,4,4,4,4
East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong,4,4,4,4,4,4,4,4


In [31]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(vancouver_venues['Venue Category'].unique())))

There are 98 uniques categories.


In [32]:
#print out the list of categories
vancouver_venues['Venue Category'].unique()[:225]

array(['Comedy Club', 'Chinese Restaurant', 'Liquor Store',
       'Bubble Tea Shop', 'Park', 'Ice Cream Shop', 'Grocery Store',
       "Women's Store", 'Diner', 'Theater', 'Coffee Shop', 'Plaza',
       'Burger Joint', 'Hotel', 'Breakfast Spot', 'Pub',
       'Vietnamese Restaurant', 'Pizza Place', 'Cosmetics Shop',
       'Dessert Shop', 'Tea Room', 'Food Truck', 'Taco Place',
       'Leather Goods Store', 'Gym', 'Café', 'Clothing Store',
       'Mediterranean Restaurant', 'Jewelry Store', 'Food Court',
       'American Restaurant', 'Fast Food Restaurant', 'Yoga Studio',
       'Australian Restaurant', 'Salad Place', 'Restaurant', 'Playground',
       'Trail', 'National Park', 'Brewery', 'Auto Garage', 'Speakeasy',
       'Japanese Restaurant', 'Gluten-free Restaurant',
       'Sushi Restaurant', 'Gift Shop', 'Harbor / Marina',
       'Shanghai Restaurant', 'Gym / Fitness Center',
       'Italian Restaurant', 'Middle Eastern Restaurant', 'Bus Stop',
       'Deli / Bodega', 'Sandwich 

***Analyze Each Neighborhood***

In [33]:
# one hot encoding
to_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Postcode'] = vancouver_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(594, 99)


,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,V2A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,V2A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,V2A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,V2A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,V6A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
to_grouped = to_onehot.groupby(["Postcode"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(132, 99)


,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio
0,V1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,V1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,V1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,V1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,V1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,V1J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,V1K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,V1L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,V1N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,V1P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Vancouver into 3 clusters.

In [35]:
# set number of clusters
toclusters = 5

to_clustering = to_grouped.drop(["Postcode"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:179]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 4, 2, 2, 0, 2, 3, 0, 2, 2,
       2, 3, 0, 2, 2, 2, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [36]:
# create a new dataframe that includes the cluster for each neighborhood.
to_merged = to_grouped.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

to_merged.head()
print(to_merged.shape)

(132, 100)


In [37]:
#add latitude/longitude for each neighborhood
to_merged = to_merged.join(df.set_index("postalCode"), on="Postcode")

print(to_merged.shape)
print(df.shape)
to_merged.head()

(132, 104)
(179, 5)


,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster Labels,neighbourhood,community,Latitude,Longitude
0,V1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,VernonEast,,49.26038,-123.11336
1,V1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Cranbrook,,49.26038,-123.11336
2,V1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Salmon Arm,,49.26038,-123.11336
3,V1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Dawson Creek,,49.26038,-123.11336
4,V1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,VernonWest,,49.26038,-123.11336


In [47]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
project.save_data(data=to_merged.to_csv(index=False),file_name='clusters.csv',overwrite=True)

(132, 104)


{'file_name': 'clusters.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'courseracapstone-donotdelete-pr-gyrtxtz7k1quxs',
 'asset_id': 'a91fb1db-7c08-4c9e-9b45-cf8522d54cbc'}

Visualize the clusters

In [39]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [53]:
# create map
map_clusters = folium.Map(location=[lat_vancouver, lon_vancouver], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Postcode'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

In [42]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster Labels,neighbourhood,community,Latitude,Longitude
0,V1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,VernonEast,,49.26038,-123.113360
104,V8R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Victoria,North Oak Bay / North Fernwood / North Jubilee...,49.26038,-123.113360
103,V8P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Victoria,SE Saanich / UVic,49.26038,-123.113360
102,V8N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Victoria,East Saanich,49.26038,-123.113360
101,V8M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Central Saanich,,49.26038,-123.113360
100,V8L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sidney,North Saanich / YYJ,49.26038,-123.113360
99,V8K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Salt Spring Island,,49.26038,-123.113360
98,V8J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Prince Rupert,,49.26038,-123.113360
97,V8H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Not assigned,,49.26038,-123.113360
96,V8G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Terrace,,49.26038,-123.113360


In [43]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster Labels,neighbourhood,community,Latitude,Longitude
81,V6W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,RichmondSoutheast,,49.14167,-123.067169


In [44]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster Labels,neighbourhood,community,Latitude,Longitude
37,V3E,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,2,CoquitlamNorth,,49.315380,-122.749502
34,V2X,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,2,Maple RidgeWest,,49.218425,-122.619979
36,V3B,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.5,0.000000,2,Port CoquitlamCentral,,49.268985,-122.753858
48,V4E,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.0,0.000000,0.000000,0.0,0.000000,2,DeltaEast,,49.116865,-122.907341
90,V7Y,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.166667,2,Vancouver,Pacific Centre,49.275253,-123.116773
49,V4G,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,2,DeltaEast Central,,49.137320,-123.028845
70,V5V,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.100000,0.0,0.000000,2,Vancouver,West Kensington-Cedar Cottage / NE Riley Park-...,49.248430,-123.090562
69,V5T,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,2,Vancouver,East Mount Pleasant,49.263660,-123.091207
67,V5P,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.333333,0.0,0.000000,2,Vancouver,SE Kensington-Cedar Cottage / Victoria-Fraserview,49.223382,-123.067150
66,V5N,0.000000,0.125,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,2,Vancouver,South Grandview-Woodland / NE Kensington-Cedar...,49.254843,-123.067400


In [45]:
#Cluster 3
to_merged.loc[to_merged['Cluster Labels'] == 3]

,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster Labels,neighbourhood,community,Latitude,Longitude
40,V3N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,Burnaby,East Big Bend / Stride Avenue / Edmonds / Cari...,49.216185,-122.944937
68,V5S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,Vancouver,Killarney,49.215345,-123.040543
45,V3V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,SurreyOuter Northwest,,49.188800,-122.889988


In [46]:
#Cluster 4
to_merged.loc[to_merged['Cluster Labels'] == 4]

,Postcode,American Restaurant,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,Auto Garage,Bakery,Bank,Bar,Beer Garden,...,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store,Yoga Studio,Cluster Labels,neighbourhood,community,Latitude,Longitude
35,V2Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4,Langley TownshipNorthwest,,49.12826,-122.620481
52,V4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4,DeltaNorthwest,,49.09506,-123.025586
